Creates the scaffold of Google sheets needed for functionalities  
Because of Google's scopes, even if you want to maintain these manually they at least need created by the API  
This will create them and save the spreadsheet ID in config, and create all the sheet names I am expecting  
Because of Google's Sheets API vs Drive API support, you can create the sheets here and file them away in your Drive where you want - they will always create in root
Spreadsheets:  
1. LocalMasterData
2. LocalTxData
3. BatchUpload
4. BatchTx

Connections need to be allowed the first time you load some sheets in the web interface ex. to get LocalMasterData in BatchUpload to use Data Validation to assist in entry

In [ ]:
import common
import json
logging = common.start_logger("BootstrapGoogleSheets",'DEBUG')
creds = common.GoogleOAuth()
mani = ['LocalMasterData','LocalTxData','BatchUpload','BatchTx']
sheets={}
common.GrConf()
interface = common.CONF.interface

In [ ]:
#Create the 3 sheets and save them to config

sheets = {}
for x in mani:
    sheet = common.create(x,creds)
    sheetDef = {'spreadsheetId':sheet}
    sheets[x]=sheetDef

file = json.load(open('localEnv.json','r'))
file['GoogleAPI']['Sheets']=sheets

write = open('localEnv.json','w')
json.dump(file,write)
write.close()

With the spreadsheet IDs in the config, create the sheets needed in each

In [ ]:
# Get all the Master Data like interfaces defined
# These are joinable Master Data I expect to use for input help in a google sheet, not all of it
#Might throw this out, who wants local amster data in a google sheet? can add a sheet into the mass update sheets to fill it on demand
# LocalMasterData targetted interfaces


masterdata = []
for x in interface['Objects']:
    try:
        if interface['Objects'][x]['sheet'] == 'masterdata':
            masterdata.append(x)
    except KeyError:
        continue

sheets['LocalMasterData']['sheets'] = masterdata
logging.debug(f'Master Data sheets Discovered: {sheets['LocalMasterData']['sheets']}')

# Get all the transaction data like interfaces defined
# Localtxdata target
txdata = []
for x in interface['Objects']:
    try:
        if interface['Objects'][x]['sheet'] == 'tx':
            txdata.append(x)
    except KeyError:
        continue

sheets['LocalTxData']['sheets'] = txdata
logging.debug(f'Transaction Data sheets discovered: {sheets['LocalTxData']['sheets']}')

# Get all the common posting routes from interfaces
# BatchUpload target
objPost = []
for x in interface['Objects']:
    if eval(interface['Objects'][x]['post']): objPost.append(x)
objPost.append('LastPost')

sheets['BatchUpload']['sheets'] = objPost
logging.debug(f'Postable Object sheets discovered: {sheets['BatchUpload']['sheets']}')

# Get all the common tx routes from itnerfaces
# BatchTx target
txPut = []
for txobject in interface['Tx']:
    for txtype in interface['Tx'][txobject]['txType']:
        txPut.append(f'{txobject}_{txtype}')
txPut.append('LastPost')

sheets['BatchTx']['sheets']=txPut
logging.debug(f'Transaction sheets discovered: {sheets['BatchTx']['sheets']}')

In [ ]:
# Start building the API requests for the sheets
# Loop through all files adding each sheet
respLog = []
for x in mani:
    logging.info(f'Starting sheet run for: {file['GoogleAPI']['Sheets'][x]}')
    updater = common.GoBatchUpdate(creds,file['GoogleAPI']['Sheets'][x]['spreadsheetId'])
    iSheet = sheets[x]['sheets']

    for y in iSheet:
        diff = {
        "addSheet": {
            "properties": {
                "title": y
                }
            }
        }
        if y == 'LastPost': # Tag LastPost as a special tab with blue color
            specialColor = {"rgbColor":{
              "red": 0.212,
              "green": 0.549,
              "blue": 0.8,
              "alpha": 0.722}
                           }
            diff['addSheet']['properties']["tabColorStyle"] = specialColor                
        updater.addReq(diff)
        
    updater.call()
    #print(updater.r)
    #temporary measure while I write something to parse response JSONs to the config file
    respLog.append(updater.r)

In [ ]:
#regularly scheduled program: parse results and save sheetId for quick reference
file = json.load(open('localEnv.json','r'))
fileJIC = file
# file = fileJIC #break in case of emergency
#unwind response list and pull names and IDs
for x in respLog:
    match = False
    for y in file['GoogleAPI']['Sheets']:
        if file['GoogleAPI']['Sheets'][y]['spreadsheetId'] == x['spreadsheetId']: #first match toplevels at spreadsheet ID
            sheetDict={}
            for z in x['replies']: #pull each name and sheet ID into a dict to file under the config sheet name
                sheetDict[z['addSheet']['properties']['title']] = z['addSheet']['properties']['sheetId']
            file['GoogleAPI']['Sheets'][y]['sheets']=sheetDict #stick the dict in its home
            match = True #single match found, no use keeping up
    if match: continue


write = open('localEnv.json','w')
json.dump(file,write)
write.close()

In [ ]:
# Clean up sheet 1 in each spreadsheet
for x in mani:
    logging.info(f'Starting sheet delete 1 for: {file['GoogleAPI']['Sheets'][x]}')
    updater = common.GoBatchUpdate(creds,file['GoogleAPI']['Sheets'][x]['spreadsheetId'])
    diff={"deleteSheet":{"sheetId": 0}}
    updater.addReq(diff)
    updater.call()

In [ ]:
# Add headers to 
# LocalMasterData
# LocalTxData
# based on interface calls
# ignoring user fields for now i.e. they will show up as a string form JSON list?
# first row is reserved for utilities ex. links back to jupyter server or other webhooks

#Lower priority - use pandas and other py options

In [ ]:
# Add headers to 
# BatchUpload
# based on interface calls
# ignoring user fields for now i.e. they will show up as a string form JSON list?
# first row is reserved for utilities ex. links back to jupyter server or other webhooks

#Additional field: "Ready" which will help wrangle previously retrieved data
file = json.load(open('localEnv.json','r',encoding="utf8"))
spreadsheet = file['GoogleAPI']['Sheets']['BatchUpload']['spreadsheetId']
header = []
for x in file['GoogleAPI']['Sheets']['BatchUpload']['sheets']:
    #Get an example
    try:
        iterCaller = common.GrGetAPI('Objects',x)
    except: continue
    iterCaller.buildFreeParam({'limit':1})
    iterCaller.get()
    if "200" not in str(iterCaller.r):
        logging.error(iterCaller.r+' API failure')
        raise RuntimeError(iterCaller.r)
        continue
    try:
        example = iterCaller.r.json()
        example = list(example[0].keys())
    except:
        # Hide unused tabs
        updater = common.GoBatchUpdate(creds,spreadsheet)
        diffRequest = {
            "updateSheetProperties": {
              "properties": {
                  "sheetId": file['GoogleAPI']['Sheets']['BatchUpload']['sheets'][x],
                  "hidden": True,
                  "index":len(file['GoogleAPI']['Sheets']['BatchUpload']['sheets'])
              },
              "fields": "hidden,index"
            }
        }
        updater.addReq(diffRequest)
        updater.call()
        logging.info(updater.r)
        continue
    # Example will hold our header. Build a google request to stick it into B1
    header = example
    header.append('Ready')
    
    #Some housekeeping for format requests later - find special header values to color
    colorNoEdit = ['id','userfields','row_created_timestamp'] #adapt from config DDIC function - I need to add userfields in a DDIC at some point
    colorToGo = ['Ready'] #hardcode 'Ready' - should be 1 and done
    colorNoEditA1 = []
    colorToGoA1 = []
    for field in colorNoEdit:
        try:
            col = header.index(field) + 1
        except ValueError: continue
        row = 2
        colorNoEditA1.append(common.rowcol_to_a1(row,col))
    for field in colorToGo:
        try:
            col = header.index(field) + 1
        except ValueError: continue
        row = 2
        colorToGoA1.append(common.rowcol_to_a1(row,col))
        
    #ready for the data request
    data = [header]
    sheet = x
    start = 'A2'
    end = common.a1_from_table(start,data)
    logging.info(common.sheets_update(creds,spreadsheet,sheet,f"{start}:{end}",data))

    
    sheet = file['GoogleAPI']['Sheets']['BatchUpload']['sheets'][sheet]
    updater = common.GoBatchUpdate(creds,spreadsheet)
    freezeRequest = {"updateSheetProperties": {
        "properties": {
            "sheetId": sheet,
            "gridProperties": {
                "frozenRowCount": 2
            }
        },
        "fields": "gridProperties.frozenRowCount"
    }
                    }
    updater.addReq(freezeRequest)
    for A1cell in colorNoEditA1:
        row,col = common.a1_to_rowcol(A1cell)
        colorRequest = {
            "repeatCell": {
                "range": {
                    "sheetId": sheet,
                    "startRowIndex": row-1,
                    "endRowIndex": row,
                    "startColumnIndex":col-1,
                    "endColumnIndex":col
                },
                "cell": common.CONF.styleguide['cell_styles']['no_editH']['cell'],
                "fields": common.CONF.styleguide['cell_styles']['no_editH']['fields']
            }            
          }
        updater.addReq(colorRequest)
    for A1cell in colorToGoA1:
        row,col = common.a1_to_rowcol(A1cell)
        colorRequest = {
            "repeatCell": {
                "range": {
                    "sheetId": sheet,
                    "startRowIndex": row-1,
                    "endRowIndex": row,
                    "startColumnIndex":col-1,
                    "endColumnIndex":col
                },
                "cell":  common.CONF.styleguide['cell_styles']['onswitchH']['cell'],
                "fields": common.CONF.styleguide['cell_styles']['onswitchH']['fields']
            }            
          }
        updater.addReq(colorRequest)
    updater.call()
    logging.info(updater.r)



In [ ]:
# Add headers to 
#BatchTx
#based on schema kept in config file
# Need to reorder for human readable?

file = json.load(open('localEnv.json','r',encoding="utf8"))
spreadsheet = file['GoogleAPI']['Sheets']['BatchTx']['spreadsheetId']
header = []
for sheet in file['GoogleAPI']['Sheets']['BatchTx']['sheets']:
    try:
        obj,tx = sheet.split("_")
    except ValueError:
        continue
    header = list(file['Interface']['Tx'][obj]['txType'][tx].keys())
    header.sort()
    #sorting and relevance is implied in first 2 characters - drop XX prefixed (not display relevant) and slice it out now that its sorted
    cleanHeader = []
    for name in header:
        try:
            int(name[:2])
            cleanHeader.append(name)
        except ValueError: continue
    header = [name[2:] for name in cleanHeader]
    header.append("Ready")

    colorNoEdit = common.CONF.list_uneditable() #adapt from config DDIC function - I need to add userfields in a DDIC at some point
    colorToGo = ['Ready'] #hardcode 'Ready' - should be 1 and done
    colorNoEditA1 = []
    colorToGoA1 = []
    for field in colorNoEdit:
        try:
            col = header.index(field) + 1
        except ValueError: continue
        row = 2
        colorNoEditA1.append(common.rowcol_to_a1(row,col))
    for field in colorToGo:
        try:
            col = header.index(field) + 1
        except ValueError: continue
        row = 2
        colorToGoA1.append(common.rowcol_to_a1(row,col))
        
    #ready for the data request
    data = [header]
    start = 'A2'
    end = common.a1_from_table(start,data)
    logging.info(common.sheets_update(creds,spreadsheet,sheet,f"{start}:{end}",data))
    updater = common.GoBatchUpdate(creds,spreadsheet)
    sheet = file['GoogleAPI']['Sheets']['BatchTx']['sheets'][sheet]
    freezeRequest = {"updateSheetProperties": {
        "properties": {
            "sheetId": sheet,
            "gridProperties": {
                "frozenRowCount": 2
            }
        },
        "fields": "gridProperties.frozenRowCount"
    }
                    }
    updater.addReq(freezeRequest)
    for A1cell in colorNoEditA1:
        row,col = common.a1_to_rowcol(A1cell)
        colorRequest = {
            "repeatCell": {
                "range": {
                    "sheetId": sheet,
                    "startRowIndex": row-1,
                    "endRowIndex": row,
                    "startColumnIndex":col-1,
                    "endColumnIndex":col
                },
                "cell": common.CONF.styleguide['cell_styles']['no_editH']['cell'],
                "fields": common.CONF.styleguide['cell_styles']['no_editH']['fields']
            }            
          }
        updater.addReq(colorRequest)
    for A1cell in colorToGoA1:
        row,col = common.a1_to_rowcol(A1cell)
        colorRequest = {
            "repeatCell": {
                "range": {
                    "sheetId": sheet,
                    "startRowIndex": row-1,
                    "endRowIndex": row,
                    "startColumnIndex":col-1,
                    "endColumnIndex":col
                },
                "cell":  common.CONF.styleguide['cell_styles']['onswitchH']['cell'],
                "fields": common.CONF.styleguide['cell_styles']['onswitchH']['fields']
            }            
          }
        updater.addReq(colorRequest)
    updater.call()
    logging.info(updater.r)